# Synchronization of Coupled Oscillators and Synchronization in Nature

###### Kergan Sanderson, NCSU PY 251 Spring 2023

This notebook is the culmination of a semester of Python programming lessons and assignments for PY 251. This project applies methods and teachings from the semester to numerically represent a system of coupled oscillators. Data analysis is performed on images to show the presence of synchronization in natural systems.

### Synchronized Pendulums

First observed by Christiaan Huygens in the 17th century, the synchronization of pairs of moving pendulums has been further researched and questioned over the past few centuries.

Huygens was working on making pendulum clocks applicable for telling time onboard ships traveling across seas to be used in determining longitude. While the pendulum clock method for telling location would work, it was very hard to create two pendulum clocks with similar time keeping ability due to inaccurate measurements. While attempting to solve this problem, Huygens discovered that hanging two pendulums from the same beam resulted in the pendulums eventually synchronizing in phase after a span of time.

While pendulum clocks did not prove to be a reliable solution to the longitude problem, the observations of systems of synchronizing pendulums became areas of research for scientists over the next few centuries.

Models were created for systems of coupled oscillators, one of which is used in this project. Systems of coupled pendulums are still under study to this day due to the many component interactions between the oscillators and the complexity of creating a model that encapsulates all of the energy transfers that occur.

### Kuramoto's Model

Proposed by Yoshi Kuramoto, a Japanese Physicist in the Nonlinear Dynamics group at Kyoto University, this model is used to describe sets of coupled oscillators.

$$ \frac{d \theta_i}{dt} = \omega_i + \frac{K}{N} \sum^N_{j=1} \sin (\theta_j - \theta_i), i = 1 ... N$$

In this form of the Kuramoto model, each oscillator has its own natural frequency ($\omega_i$). This value is combined with the difference in phase from all other oscillators in the system. The coupling strength $K$ is a value representing how influential the oscillators are on each other.

### This Project

There are two parts to this notebook. The first is an application of the Kuramoto Model with interactive widgets and a visual output of the simulation. The second is a report on the use of data analysis techniques on the synchronization of large groups of fireflies.

## The Simulation

Interactive widgets are used to modify the number of oscillators in the system, the coupling strength between the oscillators, and the natural frequency of each oscillator. This section of code sets up those widgets so that they can record user input.

### To Use:

1. Run Cell 1
    - Change the four sliders to your liking
2. Run Cell 2
    - Modify the parameters to your liking
3. Run Cell 3
4. Run Cell 4
    - Turn looping on
    - Hit play
    - Use the slider to modify the value of K to see the effect

# Cell 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# the ipywidgets package provides interactive widgets for Jupyter Notebook
import ipywidgets as widgets
# display is used to avoid widgets from being cleared by other output
from IPython.display import display

g = 9.8 # m/s/s

# creating the widgets for the number of oscillators and the coupling strength
numosc = widgets.IntSlider(value=5,min=0,max=10,step=1,description='# Oscillators')
Kval = widgets.FloatSlider(value=1,min=0,max=5,step=0.01,description='K value')
tmaxval = widgets.FloatSlider(value=2,min=1,max=2,step=0.01,description='Time End')
# change mult to higher integers for larger animation intervals
multiplier = widgets.IntSlider(value=3,min=1,max=5,step=1,description='Anim Speed')
display(numosc)
display(Kval)
display(tmaxval)
display(multiplier)

# Cell 2

In [ ]:
# creating the widgets for the individual pendulum arm lengths.
osciwgtsl = [widgets.FloatSlider(value=np.random.rand()*20,min=1,max=20,step=0.1,description='#'+str(i+1),
                                  orientation='vertical') for i in range(numosc.value)]
print("Length of each pendulum arm in meters:")
display(widgets.Box(osciwgtsl))

# creating the widgets for the individual oscillator initial angles.
osciwgtsa = [widgets.FloatSlider(value=np.random.rand()*2*np.pi,min=0,max=2*np.pi,description='#'+str(i+1),
                                  orientation='vertical') for i in range(numosc.value)]
print("Initial angle of each oscillator in radians:")
display(widgets.Box(osciwgtsa))

# once this box is run, the number of oscillators is fixed as N
N = numosc.value
mult = multiplier.value

The following section of code sets up a class for the oscillators.

# Cell 3

In [ ]:
class Oscillator:
    def __init__(self,name,length,angle):
        self.name = name
        self.w = np.sqrt(g/length)
        self.angle = angle
        self.history=[]
        self.history.append(self.angle)
    
    def osciadd(self,coupledinfluence,dt):
        self.angle += (self.w + coupledinfluence)*dt
        self.history.append(self.angle)

The next block of code is the simulation itself. Make any desired changes to the parameters before executing this block.

# Cell 4

In [ ]:
# initializing the oscillators
oscis=[]
for i in range(N):
    oscis.append('Osci'+str(i+1))
    oscis[i] = Oscillator(oscis[i],osciwgtsl[i].value,osciwgtsa[i].value)

# create the timescale
tmin = 0 #s
tmax = tmaxval.value #s
nts = 100
t = np.linspace(tmin,tmax,nts)
dt = t[1]-t[0]

# this function evaluates the Kuramoto Model for all oscillators for a single dt step
def Kuramoto():
    kura = np.zeros(N)
    
    for i in range(N):
        for j in range(N):
            kura[i] += np.sin((oscis[j].angle%(2*np.pi))-(oscis[i].angle%(2*np.pi)))
    
    kura = kura*Kval.value/N

    for i in range(N):
        oscis[i].osciadd(kura[i],dt)

# this function is used in the Play widget to re-calculate and replot every frame
def AniKuramoto(n=0):
    # time array is shifted by an increment every frame
    t = np.linspace(tmin+mult*n*dt,tmax+mult*n*dt,nts)
    
    for i in range(nts-1):
        Kuramoto()

    for i in range(N):
        plt.plot(t,np.sin(oscis[i].history),label=oscis[i].name)
    plt.title("sin(Angle of Oscillator) vs. Time for Chosen Parameters")
    plt.ylabel("sin(angle of oscillator)")
    plt.xlabel("Time (s)")
    # plot limit is shifted by the same limit to keep the plot centered on the data
    plt.xlim(mult*n*dt,(mult*n*dt)+tmax)
    plt.ylim(-1.25,1.25)
    plt.legend(bbox_to_anchor=(1,1), loc="upper left")
    
    # after each plot, mult*1 takes the second angle and makes it the first for the next frame
    for i in range(N):
        oscis[i].angle = oscis[i].history[mult*1]
        oscis[i].history = []
        oscis[i].history.append(oscis[i].angle)

widgets.interact(AniKuramoto, n=widgets.Play(min=0,max=nts-1,step=1,interval=250))
display(Kval)

### Interpreting the Plot

The above plot does not directly show the positions of the pendulums. Kuramoto's model is commonly demonstrated with a set of dots (oscillators) orbiting around a center point, all going the same direction. These oscillators travel at their respective frequencies in addition to the shifts caused by the coupling with all the other oscillators in the circle of travel.

This plot shows the sine of the phase position of each of the pendulums. One way to interpret this is that a value of -1 means that the pendulum is at the left side of its swing and a value of +1 means that the pendulum is at the right side of its swing.

For high values of K, the oscillators synchronize in frequency very quickly and are very close in phase with each other. For low values of K, the oscillators either take a while to synchronize or do not exhibit a clear pattern of synchronization. In the latter scenario, the oscillators are not coupled strongly enough to present a clear relationship.

***

## Synchronization of Fireflies

Systems of synchronizing oscillators are created by people. Coupled pendulum clocks and metronomes on soda cans are systems that are created and then studied by humans. It may be surprising that synchronization is not just limited to mechanical systems. Examples of synchronization can be observed in human group behavior, as when an audience that is clapping begins to synchronizing in clapping rhythm of when people walking together get in-step with each other. It is for this second reason that military troops 'break step' (walk with different timing) when crossing bridges in order to avoid stressing the bridge.

Synchronization is not limited to human activity. In this section of the project, code is written to analyze a segment of a video that shows the behavior of fireflies. In the video, fireflies can be seen flashing in sync with one another, often in very large groups.

This section uses image processing tools to count the number of fireflies that are flashing in sync with each other at a regular interval as time progresses.

### Obtaining the Images

This block of code is used to process a set of images taken from [this video](https://www.youtube.com/watch?v=ZGvtnE1Wy6U&t=64s). This video was part of an installation titled "Synchronicity", which was shown in Basel, Switzerland in 2015. The video has several segments which show different groups of fireflies synchronizing. This was part of an experiment where LEDs that flashed at regular time intervals were placed in groups of fireflies to stimulate them and cause them to synchronize more quickly than if they were on their own.

To obtain the images, I slowed the playback speed of the video so that I could more precisely pause the video and take a screenshot of the moment when the majority of fireflies were flashing. Each image was processed and the number of fireflies present in the image was recorded. After all the images in the sequence were processed, the values were plotted in chronological order on a plot to observe any trends in the number of fireflies that were synchronized.

Photos were taken from timestamps 00:52 to 00:58 in the video.

Example image (the first flash):

![image](Images/Fireflies1.png)

Twelve of these images were taken, one for the largest group during each flash.

The below code can be run to produce the plot of the number of fireflies in each of the 12 images.

# Cell 5

In [ ]:
import skimage                  # for image-processing
from skimage import io          # give ourselves a shortcut 
from skimage import measure     # give ourselves a shortcut 

import numpy as np
import matplotlib.pyplot as plt

# ff for fireflies
ffnames = []
ffimages = []

for i in range(12):
    ffnames.append('Fireflies'+str(i+1))
    ffimages.append(io.imread('Images/'+str(ffnames[i])+'.png'))

imagenums = np.linspace(1,12,12)
numff = np.zeros(12)

for i in range(12):
    ffimages[i] = ffimages[i][:,:,1] # using only the green channel
    threshold = ffimages[i] > 10*ffimages[i].mean() # only brightest pixels
    all_labels = measure.label(threshold) # label the regions
    regions = measure.regionprops(all_labels) # store properties of all_labels
    numff[i] = len(regions) # number of regions in the current image

plt.plot(imagenums,numff)
plt.title('Number of Fireflies in largest synchronized group for 12 consecutive flashes')
plt.ylabel('Number of Fireflies')
plt.xlabel('Flash Number')
plt.ylim(0,150)
plt.show

### Interpreting the Plot

This plot does not trend in an upward or downward direction. In this section of the video, the fireflies are already fairly in sync with each other. This plot shows that the number of fireflies in sync stays fairly constant. Some fireflies may join in while others may fall out of sync with the main group, but the number of fireflies in sync does not deviate far from the previous numbers.

In this video, not all of the fireflies were synchronized. This plot does not show all of the fireflies that were flashing in the video. There were two main groups of fireflies, one larger and one smaller. The larger group remained constant in size for the 5 second span of time that was shown.

***

## Online Resources Used

- [Inspiration for Play Widget Animation from Kimberly Fessel's Youtube Channel](https://www.youtube.com/watch?v=0bNh3kbONXo)
- [Jupyter Widgets Documentation: Simple Widget Introduction](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html)
- [Jupyter Widgets Documentation: Play Widget](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#play-animation-widget)
- [Jupyter Widgets Documentation: Interact](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html)
- [Solution for Putting a matplotlib.pyplot Legend Outside of the Plot](https://www.statology.org/matplotlib-legend-outside-plot/#:~:text=Often%20you%20may%20want%20to,combined%20with%20the%20bbox_to_anchor%20argument)

Most of the techniques and methods used in this project came from the modules that were completed as part of the PY 251 course at North Carolina State University.

***

Kergan Sanderson, 2023